<img src="logo.png" alt="drawing" width="500"/>

## Python Data Engineering Challenge

Our system ingests search term data from Google Ads API into a PostgreSQL database, via an AWS S3 Data Lake.

Once ingested we score each search term with its Return On Ad Spend (ROAS).

```text
ROAS = conversion value / cost
```


### Task

Three CSVs have been given - campaigns.csv, adgroups.csv and search_terms.csv. 

First ingest these 3 CSVs into a database, ensure the data ingestion is idempotent. 

Secondly, the adgroup alias is in the format:

`Shift - Shopping - <country> - <campaign structure value> - <priority> - <random string> - <hash>`

We sometimes need to know the ROAS aggregated by `country` and/or by `priority`. 

Build something to allow for those aggregations to be queried easily.


### Submission

Please fork this repo to complete the challenge, once done email back link to your repo.

Good luck we are rooting for you!

In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse as up
import sql_server_creds

In [2]:
# Initialize postgresql server and engine
up.uses_netloc.append("postgres")
url = up.urlparse(sql_server_creds.url)
engine = create_engine(\
    f"postgresql+psycopg2://{url.username}:{url.password}@{url.hostname}/{url.path[1:]}?client_encoding=utf8")

In [3]:
# Import the source data files
adgroups_df = pd.read_csv("adgroups.csv")
campaigns_df = pd.read_csv("campaigns.csv")
search_terms_df = pd.read_csv("search_terms.csv")

# Create their tables (idempotently)
adgroups_df.to_sql('adgroups_tbl', engine, index=False, if_exists='replace')
campaigns_df.to_sql('campaigns_tbl', engine, index=False, if_exists='replace')
search_terms_df.to_sql('search_terms_tbl', engine, index=False, if_exists='replace')

In [4]:
adgroups_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221855 entries, 0 to 221854
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ad_group_id  221855 non-null  int64 
 1   campaign_id  221855 non-null  int64 
 2   alias        221855 non-null  object
 3   status       221855 non-null  object
dtypes: int64(2), object(2)
memory usage: 6.8+ MB


In [5]:
campaigns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221855 entries, 0 to 221854
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   campaign_id      221855 non-null  int64 
 1   structure_value  221855 non-null  object
 2   status           221855 non-null  object
dtypes: int64(1), object(2)
memory usage: 5.1+ MB


In [6]:
search_terms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221855 entries, 0 to 221854
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   date              221855 non-null  object 
 1   ad_group_id       221855 non-null  int64  
 2   campaign_id       221855 non-null  int64  
 3   clicks            221855 non-null  int64  
 4   cost              221855 non-null  float64
 5   conversion_value  221855 non-null  float64
 6   conversions       221855 non-null  int64  
 7   search_term       221855 non-null  object 
dtypes: float64(2), int64(4), object(2)
memory usage: 13.5+ MB


In [7]:
# Remove pandas truncation
pd.set_option('display.max_colwidth', None)

In [8]:
# View top 10 of the tables
pd.read_sql("select * from adgroups_tbl limit 10", engine)

,ad_group_id,campaign_id,alias,status
0,66372665454,1578451881,Shift - Shopping - GB - venum - LOW - monkey-ack-robert-comet - 817ce4882dfc499886ca8670ccd5cbf9,REMOVED
1,84481260174,1578451584,Shift - Shopping - GB - ellesse - HIGH - oscar-gee-princess-mexico - d77d4e4c99a4462991dd51ae0051e039,ENABLED
2,59624654596,1578451386,Shift - Shopping - GB - Converse - HIGH - failed-five-cola-mockingbird - 7f9a1fbba2e14a4e95bb7375181f0c9b,REMOVED
3,59977520149,1578412457,Shift - Shopping - GB - Wilson - HIGH - vermont-oregon-oscar-uncle - 611936a2347d4da3b2fb4aabe8f8ff0a,REMOVED
4,102171970298,9872103720,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED
5,102171970298,9872103720,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED
6,58309319903,1578451386,Shift - Shopping - GB - Converse - HIGH - three-equal-berlin-iowa - 6ddac5adf5ac4851bd9ed4e3108fd22b,REMOVED
7,119487436958,1578451623,Shift - Shopping - GB - spalding - HIGH - louisiana-april-nine-east - 5cf1d60650ec4ef58a72137040d58b26,ENABLED
8,59617309655,1578451386,Shift - Shopping - GB - Converse - HIGH - summer-whiskey-princess-september - 38a70b420a5f42e4a30d5f42e1620e4c,REMOVED
9,102171970298,9872103720,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED


Secondly, the adgroup alias is in the format:

`Shift - Shopping - <country> - <campaign structure value> - <priority> - <random string> - <hash>`

We sometimes need to know the ROAS aggregated by `country` and/or by `priority`. 

Build something to allow for those aggregations to be queried easily.

In [9]:
pd.read_sql("select ad_group_id, campaign_id,\
            split_part(alias,' - ',3) country,\
            split_part(alias,' - ',5) priority,\
            alias, status from adgroups_tbl limit 10", engine)

,ad_group_id,campaign_id,country,priority,alias,status
0,66372665454,1578451881,GB,LOW,Shift - Shopping - GB - venum - LOW - monkey-ack-robert-comet - 817ce4882dfc499886ca8670ccd5cbf9,REMOVED
1,84481260174,1578451584,GB,HIGH,Shift - Shopping - GB - ellesse - HIGH - oscar-gee-princess-mexico - d77d4e4c99a4462991dd51ae0051e039,ENABLED
2,59624654596,1578451386,GB,HIGH,Shift - Shopping - GB - Converse - HIGH - failed-five-cola-mockingbird - 7f9a1fbba2e14a4e95bb7375181f0c9b,REMOVED
3,59977520149,1578412457,GB,HIGH,Shift - Shopping - GB - Wilson - HIGH - vermont-oregon-oscar-uncle - 611936a2347d4da3b2fb4aabe8f8ff0a,REMOVED
4,102171970298,9872103720,GB,HIGH,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED
5,102171970298,9872103720,GB,HIGH,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED
6,58309319903,1578451386,GB,HIGH,Shift - Shopping - GB - Converse - HIGH - three-equal-berlin-iowa - 6ddac5adf5ac4851bd9ed4e3108fd22b,REMOVED
7,119487436958,1578451623,GB,HIGH,Shift - Shopping - GB - spalding - HIGH - louisiana-april-nine-east - 5cf1d60650ec4ef58a72137040d58b26,ENABLED
8,59617309655,1578451386,GB,HIGH,Shift - Shopping - GB - Converse - HIGH - summer-whiskey-princess-september - 38a70b420a5f42e4a30d5f42e1620e4c,REMOVED
9,102171970298,9872103720,GB,HIGH,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED


In [10]:
# Create a new table with the country and priority fields populated
pd.read_sql("drop table if exists adgroups_final_tbl;\
            create table adgroups_final_tbl as \
            select ad_group_id, campaign_id,\
            split_part(alias,' - ',3) country,\
            split_part(alias,' - ',5) priority,\
            alias, status from adgroups_tbl;\
            select * from adgroups_final_tbl limit 10", engine)

,ad_group_id,campaign_id,country,priority,alias,status
0,66372665454,1578451881,GB,LOW,Shift - Shopping - GB - venum - LOW - monkey-ack-robert-comet - 817ce4882dfc499886ca8670ccd5cbf9,REMOVED
1,84481260174,1578451584,GB,HIGH,Shift - Shopping - GB - ellesse - HIGH - oscar-gee-princess-mexico - d77d4e4c99a4462991dd51ae0051e039,ENABLED
2,59624654596,1578451386,GB,HIGH,Shift - Shopping - GB - Converse - HIGH - failed-five-cola-mockingbird - 7f9a1fbba2e14a4e95bb7375181f0c9b,REMOVED
3,59977520149,1578412457,GB,HIGH,Shift - Shopping - GB - Wilson - HIGH - vermont-oregon-oscar-uncle - 611936a2347d4da3b2fb4aabe8f8ff0a,REMOVED
4,102171970298,9872103720,GB,HIGH,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED
5,102171970298,9872103720,GB,HIGH,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED
6,58309319903,1578451386,GB,HIGH,Shift - Shopping - GB - Converse - HIGH - three-equal-berlin-iowa - 6ddac5adf5ac4851bd9ed4e3108fd22b,REMOVED
7,119487436958,1578451623,GB,HIGH,Shift - Shopping - GB - spalding - HIGH - louisiana-april-nine-east - 5cf1d60650ec4ef58a72137040d58b26,ENABLED
8,59617309655,1578451386,GB,HIGH,Shift - Shopping - GB - Converse - HIGH - summer-whiskey-princess-september - 38a70b420a5f42e4a30d5f42e1620e4c,REMOVED
9,102171970298,9872103720,GB,HIGH,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED


In [11]:
pd.read_sql("select * from campaigns_tbl limit 10", engine)

,campaign_id,structure_value,status
0,1578451881,venum,ENABLED
1,1578451584,ellesse,ENABLED
2,1578451386,converse,ENABLED
3,1578412457,wilson,ENABLED
4,9872103720,wham-o,ENABLED
5,9872103720,wham-o,ENABLED
6,1578451386,converse,ENABLED
7,1578451623,spalding,ENABLED
8,1578451386,converse,ENABLED
9,9872103720,wham-o,ENABLED


In [12]:
pd.read_sql("select * from search_terms_tbl limit 10", engine)

,date,ad_group_id,campaign_id,clicks,cost,conversion_value,conversions,search_term
0,2019-05-22,66372665454,1578451881,2,0.28,0.0,0,venum spats
1,2020-04-16,84481260174,1578451584,1,0.05,0.0,0,camillaw
2,2020-01-14,59624654596,1578451386,1,0.09,0.0,0,dc comic converse
3,2020-04-26,59977520149,1578412457,1,0.05,0.0,0,a1030
4,2020-05-30,102171970298,9872103720,1,0.17,0.0,0,h20 go
5,2020-05-30,102171970298,9872103720,1,0.11,0.0,0,asda waterslide mats
6,2019-04-23,58309319903,1578451386,1,0.01,0.0,0,cheapest infants glitter converse ebay
7,2021-06-08,119487436958,1578451623,2,0.16,0.0,0,the best basketball
8,2019-02-26,59617309655,1578451386,1,0.06,0.0,0,iridescent converse white
9,2020-05-27,102171970298,9872103720,1,0.11,0.0,0,slide mat


Our system ingests search term data from Google Ads API into a PostgreSQL database, via an AWS S3 Data Lake.

Once ingested we score each search term with its Return On Ad Spend (ROAS).

ROAS = conversion value / cost

In [13]:
pd.read_sql("select *, case when conversion_value = '0' or cost = '0' then 0 else conversion_value/cost end as ROAS\
            from search_terms_tbl where conversion_value > 0 limit 10", engine)

,date,ad_group_id,campaign_id,clicks,cost,conversion_value,conversions,search_term,roas
0,2019-12-15,75104068463,2069163253,2,0.22,21.97,1,sorbothane insoles,99.863636
1,2020-08-29,108117188792,1578411800,2,0.03,25.49,1,nike air elemental backpack,849.666667
2,2021-11-20,132703746474,1578411800,2,0.35,50.98,1,grey & black court borough low 2 trainers junior,145.657143
3,2021-01-03,111186130995,1578412289,1,0.01,24.98,1,speedo fit panel jammer,2498.000000
4,2020-06-04,60822241096,1578451368,1,0.06,12.98,1,dan carter kicking tee,216.333333
5,2021-12-22,123709863244,1578630361,1,0.07,34.98,1,puma future z white,499.714286
6,2019-05-03,59617503095,1578630361,2,0.18,27.98,1,borussia dortmund shirts cheap,155.444444
7,2021-12-06,130377149178,1578411800,1,0.27,25.98,1,kids nike hat,96.222222
8,2020-05-30,59933783979,1578451617,1,0.26,25.98,1,wilson basketball,99.923077
9,2020-09-17,79717131294,1578451617,2,0.14,25.98,1,nba basketball,185.571429


In [14]:
pd.read_sql("select distinct status from campaigns_tbl", engine)

,status
0,ENABLED
1,REMOVED


In [15]:
# Combine tables
pd.read_sql("drop table if exists combined_tbl;\
            create table combined_tbl as (\
                select a.ad_group_id, a.campaign_id, a.country, a.priority, a.alias, a.status ad_status,\
                c.structure_value, c.status campaign_status,\
                s.date search_date, s.clicks, s.cost, s.conversion_value, s.conversions, s.search_term,\
                case when s.conversion_value = '0' or s.cost = '0' then 0 else s.conversion_value/s.cost end as ROAS\
                from (select distinct * from adgroups_final_tbl) a\
                left join (select distinct * from campaigns_tbl) c on a.campaign_id = c.campaign_id\
                left join (select distinct * from search_terms_tbl) s on a.ad_group_id = s.ad_group_id\
            );\
            select * from combined_tbl limit 10", engine)

,ad_group_id,campaign_id,country,priority,alias,ad_status,structure_value,campaign_status,search_date,clicks,cost,conversion_value,conversions,search_term,roas
0,55063206410,1578451878,GB,HIGH,Shift - Shopping - GB - Under Armour - HIGH - virginia-early-kansas-july - c3df7f93eaa8473380b3441837495b48,REMOVED,under armour,ENABLED,2018-12-12,2,0.05,0.0,0,under armour sports bra,0.0
1,55063210970,1578451878,GB,HIGH,Shift - Shopping - GB - Under Armour - HIGH - six-friend-lithium-yankee - 3eb55e1dad384683942a34f1d44b781b,REMOVED,under armour,ENABLED,2019-03-22,1,0.04,0.0,0,191168364896,0.0
2,55063210970,1578451878,GB,HIGH,Shift - Shopping - GB - Under Armour - HIGH - six-friend-lithium-yankee - 3eb55e1dad384683942a34f1d44b781b,REMOVED,under armour,ENABLED,2020-05-17,1,0.03,0.0,0,women's pink under armour t shirt,0.0
3,55063212890,1578451878,GB,HIGH,Shift - Shopping - GB - Under Armour - HIGH - mississippi-dakota-colorado-arkansas - 11098a1f131b4166a42478d26a7fcad6,ENABLED,under armour,ENABLED,2020-09-28,1,0.03,0.0,0,boys under armour coldgear,0.0
4,55063212890,1578451878,GB,HIGH,Shift - Shopping - GB - Under Armour - HIGH - mississippi-dakota-colorado-arkansas - 11098a1f131b4166a42478d26a7fcad6,ENABLED,under armour,ENABLED,2020-09-28,1,0.02,0.0,0,under armour youth coldgear leggings,0.0
5,55067334650,1578451386,GB,HIGH,Shift - Shopping - GB - Converse - HIGH - delta-eleven-red-winner - 1d60a5ea0d004b7b916d387cc4ae0498,REMOVED,converse,ENABLED,2019-02-02,1,0.07,0.0,0,converse all star tracksuit mens,0.0
6,55067334650,1578451386,GB,HIGH,Shift - Shopping - GB - Converse - HIGH - delta-eleven-red-winner - 1d60a5ea0d004b7b916d387cc4ae0498,REMOVED,converse,ENABLED,2019-01-06,1,0.12,0.0,0,black converse joggers,0.0
7,55067335170,1578451386,GB,HIGH,Shift - Shopping - GB - Converse - HIGH - golf-december-cola-video - d7f8d1ab8e8d4172b857f83f377c389d,REMOVED,converse,ENABLED,2020-01-13,1,0.16,0.0,0,converse polo t shirts,0.0
8,55067336810,1578630361,GB,HIGH,Shift - Shopping - GB - Puma - HIGH - helium-mango-mirror-failed - 8735cb252c97477e81e3f31d5b7ac8a8,ENABLED,puma,ENABLED,2019-03-02,1,0.01,0.0,0,boys size 13 football boots,0.0
9,55067337770,1578630361,GB,HIGH,Shift - Shopping - GB - Puma - HIGH - berlin-oklahoma-fish-dakota - b2e76120511a41a9b27872fed1da4266,ENABLED,puma,ENABLED,2019-03-01,2,0.45,0.0,0,puma evopower vigor 3 fg,0.0


In [16]:
pd.read_sql("select * from combined_tbl where roas > 0 limit 10", engine)

,ad_group_id,campaign_id,country,priority,alias,ad_status,structure_value,campaign_status,search_date,clicks,cost,conversion_value,conversions,search_term,roas
0,55067386730,1578630361,GB,HIGH,Shift - Shopping - GB - Puma - HIGH - maryland-yankee-idaho-charlie - c115baeb6f1b46ce89a46d2107192f14,REMOVED,puma,ENABLED,2019-01-27,1,0.10,22.98,1,best green football boots kids,229.800000
1,55067402050,1578412016,GB,HIGH,Shift - Shopping - GB - Kickers - HIGH - uniform-ack-yankee-steak - 7c05a53bf6d04066876af3e2bb2591db,ENABLED,kickers,ENABLED,2019-04-16,1,0.03,24.98,1,rose gold kicker boots,832.666667
2,55067982370,1578630361,GB,HIGH,Shift - Shopping - GB - Puma - HIGH - yankee-kilo-mango-mirror - 8ec7982d71be4ef091a9269cb605f5ca,REMOVED,puma,ENABLED,2019-02-18,1,0.08,37.98,1,dortmund home top,474.750000
3,55068003730,1578411800,GB,HIGH,Shift - Shopping - GB - Nike - HIGH - black-april-ten-echo - 9ab4330b5fec4efaacc454e666bc16f5,REMOVED,nike,ENABLED,2020-05-21,1,0.01,15.98,1,kids nike sweat shorts,1598.000000
4,55068004690,1578411800,GB,HIGH,Shift - Shopping - GB - Nike - HIGH - colorado-tennessee-july-september - c1af6622db904fdfa46a8a296afb1040,REMOVED,nike,ENABLED,2019-03-30,1,0.01,12.98,1,nike jersey shorts,1298.000000
5,55070131090,1578629881,GB,LOW,Shift - Shopping - GB - adidas - LOW - lithium-lactose-lemon-chicken - b7adaacb2f38482b8fe2d0ca8c0104af,REMOVED,adidas,ENABLED,2018-11-18,1,0.06,15.98,1,adidas crib shoes,266.333333
6,55070262370,1578451158,GB,MEDIUM,Shift - Shopping - GB - Barcelona - MEDIUM - enemy-william-oscar-romeo - 31b2378b71df4d16b1270f31e2f18728,REMOVED,barcelona,ENABLED,2019-04-25,1,0.29,8.98,1,barcelona water bottle,30.965517
7,55070262370,1578451158,GB,MEDIUM,Shift - Shopping - GB - Barcelona - MEDIUM - enemy-william-oscar-romeo - 31b2378b71df4d16b1270f31e2f18728,REMOVED,barcelona,ENABLED,2018-11-30,1,0.06,8.98,1,barcelona water bottle,149.666667
8,55070262410,1578451161,GB,HIGH,Shift - Shopping - GB - Barcelona - HIGH - arizona-colorado-friend-magnesium - 80d9fdac39384c67a7944f2c08b090a8,REMOVED,barcelona,ENABLED,2019-02-21,1,0.11,8.98,1,barcelona fc drink cup,81.636364
9,55070263130,1578629887,GB,HIGH,Shift - Shopping - GB - adidas - HIGH - bakerloo-pip-happy-north - ea64a53a19184ee084d2ada2230a8433,REMOVED,adidas,ENABLED,2019-04-28,1,0.01,41.98,1,adidas havoc boxing boots kids,4198.000000
